<a href="https://colab.research.google.com/github/kmalhotra18/Multi-Agent-Deal-Finder/blob/main/ScanningAgent.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## ScanningAgent

This agent looks for promising deals by subscribing to RSS feeds.

Building an AI Powered Deal Selection System.

In [2]:
!pip install python-dotenv

In [3]:
import sys
sys.path.append("/content/drive/MyDrive/Llms/llm_engineering/week8/")

In [5]:
!pip install feedparser # package to pull RSS feeds

  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 81.3/81.3 kB 5.4 MB/s eta 0:00:00
  Created wheel for sgmllib3k: filename=sgmllib3k-1.0.0-py3-none-any.whl size=6046 sha256=c26b91b1311a2eff64c25c084e5948ad5692ed19355277a631af871edeee6bfb
  Stored in directory: /root/.cache/pip/wheels/3b/25/2a/105d6a15df6914f4d15047691c6c28f9052cc1173e40285d03
Successfully built sgmllib3k


In [26]:
!pip install --upgrade openai

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 720.4/720.4 kB 23.3 MB/s eta 0:00:00
  Attempting uninstall: openai
    Found existing installation: openai 1.78.1
    Uninstalling openai-1.78.1:
      Successfully uninstalled openai-1.78.1


In [29]:
# imports

import os
import json
from dotenv import load_dotenv
# Import AuthenticationError from openai.lib.flat_namespace
from openai import OpenAI, AuthenticationError # Import AuthenticationError directly
from agents.deals import ScrapedDeal, DealSelection

In [41]:
# Initialize and constants

import os
from openai import OpenAI
from google.colab import userdata

# Try to retrieve from Google Colab secrets first
openai_api_key = userdata.get('OPENAI_API_KEY')

if not openai_api_key:
    raise ValueError("❌ OPENAI_API_KEY not found in Colab secrets. Please add it via `Runtime > Secrets`.")

# Set for OpenAI client
os.environ['OPENAI_API_KEY'] = openai_api_key

# Initialize client
openai = OpenAI()
print("✅ OpenAI client initialized.")

openai = OpenAI()
MODEL = 'gpt-4o-mini'

✅ OpenAI client initialized.


ScrapeDeal - part of deals.py

In [8]:
# Starts by defining series of deals URLs for categories you're running

deals = ScrapedDeal.fetch(show_progress=True)

100%|██████████| 5/5 [05:02<00:00, 60.58s/it]


In [32]:
len(deals)

50

In [33]:
deals[44].describe()

'Title: Blackstone ProSeries 2-Burner 22" Propane Omnivore Griddle for $297 + free shipping\nDetails: Take $50 off the list price on this Blackstone griddle at Walmart during its 2025 Memorial Day sale. Buy Now at Walmart\nFeatures: Dual side shelves 361 sq. in. cooking surface with two heat zones 21,000 BTUs Model: 2414\nURL: https://www.dealnews.com/products/Blackstone/Blackstone-Pro-Series-2-Burner-22-Propane-Omnivore-Griddle/490028.html?iref=rss-c196'

In [34]:
system_prompt = """You identify and summarize the 5 most detailed deals from a list, by selecting deals that have the most detailed, high quality description and the most clear price.
Respond strictly in JSON with no explanation, using this format. You should provide the price as a number derived from the description. If the price of a deal isn't clear, do not include that deal in your response.
Most important is that you respond with the 5 deals that have the most detailed product description with price. It's not important to mention the terms of the deal; most important is a thorough description of the product.
Be careful with products that are described as "$XXX off" or "reduced by $XXX" - this isn't the actual price of the product. Only respond with products when you are highly confident about the price.

{"deals": [
    {
        "product_description": "Your clearly expressed summary of the product in 4-5 sentences. Details of the item are much more important than why it's a good deal. Avoid mentioning discounts and coupons; focus on the item itself. There should be a paragpraph of text for each item you choose.",
        "price": 99.99,
        "url": "the url as provided"
    },
    ...
]}"""

In [35]:
user_prompt = """Respond with the most promising 5 deals from this list, selecting those which have the most detailed, high quality product description and a clear price.
Respond strictly in JSON, and only JSON. You should rephrase the description to be a summary of the product itself, not the terms of the deal.
Remember to respond with a paragraph of text in the product_description field for each of the 5 items that you select.
Be careful with products that are described as "$XXX off" or "reduced by $XXX" - this isn't the actual price of the product. Only respond with products when you are highly confident about the price.

Deals:

"""
user_prompt += '\n\n'.join([deal.describe() for deal in deals])

In [36]:
print(user_prompt[:2000])

Respond with the most promising 5 deals from this list, selecting those which have the most detailed, high quality product description and a clear price.
Respond strictly in JSON, and only JSON. You should rephrase the description to be a summary of the product itself, not the terms of the deal.
Remember to respond with a paragraph of text in the product_description field for each of the 5 items that you select.
Be careful with products that are described as "$XXX off" or "reduced by $XXX" - this isn't the actual price of the product. Only respond with products when you are highly confident about the price. 

Deals:

Title: Onn Google TV HD Streaming Stick for $13 + free shipping w/ $35
Details: That's $2 off and the best price we've seen. Shipping is free with orders of $35 or more. Alternatively, you can choose pickup where available. Buy Now at Walmart
Features: 
URL: https://www.dealnews.com/products/Onn/Onn-Google-TV-HD-Streaming-Stick/490029.html?iref=rss-c142

Title: INIU MagPro

In [42]:
def get_recommendations():
    completion = openai.beta.chat.completions.parse(
        model="gpt-4o-mini",
        messages=[
            {"role": "system", "content": system_prompt},
            {"role": "user", "content": user_prompt}
      ],
        response_format=DealSelection
    )
    result = completion.choices[0].message.parsed
    return result

In [43]:
result = get_recommendations()

In [44]:
len(result.deals)

5

In [45]:
result.deals[1]

Deal(product_description='The INIU MagPro Slim 5,000mAh 15W Qi2 Wireless Power Bank is designed for users who need a portable charging solution on the go. This power bank features rapid PD 20W wired charging and the latest Qi2-certified 15W wireless charging technology. Additionally, it is lightweight and compact, making it easy to carry. With an industry-leading 3-year warranty and lifetime tech support, it ensures peace of mind for users.', price=35.0, url='https://www.dealnews.com/INIU-Mag-Pro-Slim-5-000-m-Ah-15-W-Qi2-Wireless-Power-Bank-for-35-free-shipping/21739977.html?iref=rss-c142')

In [46]:
from agents.scanner_agent import ScannerAgent

In [47]:
agent = ScannerAgent()
result = agent.scan()

In [50]:
result

DealSelection(deals=[Deal(product_description='The INIU MagPro Slim Power Bank offers a compact design with a 5,000mAh capacity, supporting fast 15W Qi2 wireless charging. It is equipped with a Rapid PD 20W wired charging feature, ensuring your devices are charged quickly and efficiently. This power bank also comes with a 3-year warranty and lifetime tech support, making it a reliable choice for on-the-go charging.', price=35.0, url='https://www.dealnews.com/INIU-Mag-Pro-Slim-5-000-m-Ah-15-W-Qi2-Wireless-Power-Bank-for-35-free-shipping/21739977.html?iref=rss-c142'), Deal(product_description='The Umik D3 Smart Lock features multiple access methods, including fingerprint recognition, keys, and app control for your convenience. It is designed to withstand the elements with an IP65 waterproof rating, ensuring it remains functional in various weather conditions. This smart lock is easy to install, providing enhanced security for your home without complex setup procedures.', price=79.0, url=